In [1]:
import json
import requests
!pip install requests

In [3]:
"""Available models:
- gemma3-optimized:27b
- gemma3_optimized:8bit
- gemma3:27b-it-qat
- gemma3:27b
- phi4-optimized:latest
- phi4:latest
- phi4-reasoning:plus
- qwen3_optimized:latest
- qwen3:32b
- llama3.3:latest
- llama4:scout
- mistral3.1-optimized:latest
- mistral-small3.1:latest
- qwq:latest

Notes:
1. The models marked as "optimized" are configured to use multiple GPUs for faster inference.
2. The llama4:maverick model is not available due to its size.
"""

OLLAMA_NODE = "arctrddgxa002" # TODO: Change this variable to the node where Ollama is running
BASE_URL = f"http://{OLLAMA_NODE}:11434/api/chat"

model = "gemma3-optimized:27b" # TODO: Change this variable to the model you want to use
message = "What is the capital of France?" # TODO: Change this variable to the message you want to ask the model

response = requests.post(
  BASE_URL,
  json = {
    "model": model,
    "messages": [{"role": "user", "content": message}],
    "stream": False
  }
)

print(json.dumps(response.json(), indent=2))

{
  "model": "gemma3-optimized:27b",
  "created_at": "2025-07-22T18:30:14.665993005Z",
  "message": {
    "role": "assistant",
    "content": "The capital of France is **Paris**. \n\nIt's also its largest city and a global center for art, fashion, gastronomy, and culture.\n\n\n\n"
  },
  "done_reason": "stop",
  "done": true,
  "total_duration": 42342474765,
  "load_duration": 37313181033,
  "prompt_eval_count": 16,
  "prompt_eval_duration": 836422885,
  "eval_count": 33,
  "eval_duration": 4192256587
}
